This notebook selects the subnetworks of MERIT Basin correspondingto our dams

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle, sys, os
sys.path.append("./src")

In [3]:
from config import *
from easymore_interpolation import parallel_process_grib, process_grib, pj, grib_to_nc, map_nc
from config import msm_tmp_basin_path, msm_rain_basin_path, msm_surf_path

In [4]:
outdir = "/media/tristan/Elements/Hydro/weather/MSM_basin_var/"
indir = "/media/tristan/Elements/Hydro/weather/MSM_surf/"

In [5]:
varout = varin = ['TMP_P0_L103_GLL0', 'RH_P0_L103_GLL0', 'UGRD_P0_L103_GLL0', 'VGRD_P0_L103_GLL0', 'PRES_P0_L1_GLL0',
                  'PRMSL_P0_L101_GLL0', 'TCDC_P0_L1_GLL0', 'LCDC_P0_L1_GLL0', 'MCDC_P0_L1_GLL0', 'HCDC_P0_L1_GLL0']

sup_var = ['TMP_P0_L103_GLL0_1', 'RH_P0_L103_GLL0_1', 'UGRD_P0_L103_GLL0_1', 'VGRD_P0_L103_GLL0_1', 'PRES_P0_L1_GLL0_1',
           'PRMSL_P0_L101_GLL0_1', 'TCDC_P0_L1_GLL0_1', 'LCDC_P0_L1_GLL0_1', 'MCDC_P0_L1_GLL0_1', 'HCDC_P0_L1_GLL0_1', 'APCP_P8_L1_GLL0_acc1h']

In [ ]:
parallel_process_grib(indir, outdir, varin, varout, nproc=18)

Extracting 55798 remaining files
no author name is provide and the author name is changed to (author name)!
EASYMORE is given multiple varibales to be remapped but only on format and fill valueEASYMORE repeat the format and fill value for all the variables in output files
remap file is provided; EASYMORE will use this file and skip calculation of remapping
EASYMORE will remap variable  TMP_P0_L103_GLL0  from source file to variable  TMP_P0_L103_GLL0  in remapped NeCDF file
EASYMORE will remap variable  RH_P0_L103_GLL0  from source file to variable  RH_P0_L103_GLL0  in remapped NeCDF file
EASYMORE will remap variable  UGRD_P0_L103_GLL0  from source file to variable  UGRD_P0_L103_GLL0  in remapped NeCDF file
EASYMORE will remap variable  VGRD_P0_L103_GLL0  from source file to variable  VGRD_P0_L103_GLL0  in remapped NeCDF file
EASYMORE will remap variable  PRES_P0_L1_GLL0  from source file to variable  PRES_P0_L1_GLL0  in remapped NeCDF file
EASYMORE will remap variable  PRMSL_P0_L101_GL

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



 from source file to variable  TMP_P0_L103_GLL0  in remapped NeCDF file
EASYMORE will remap variable  RH_P0_L103_GLL0  from source file to variable  RH_P0_L103_GLL0  in remapped NeCDF file
EASYMORE will remap variable  UGRD_P0_L103_GLL0  from source file to variable  UGRD_P0_L103_GLL0  in remapped NeCDF file
EASYMORE will remap variable  VGRD_P0_L103_GLL0  from source file to variable  VGRD_P0_L103_GLL0  in remapped NeCDF file
EASYMORE will remap variable  PRES_P0_L1_GLL0  from source file to variable  PRES_P0_L1_GLL0  in remapped NeCDF file
EASYMORE will remap variable  PRMSL_P0_L101_GLL0  from source file to variable  PRMSL_P0_L101_GLL0  in remapped NeCDF file
EASYMORE will remap variable  TCDC_P0_L1_GLL0  from source file to variable  TCDC_P0_L1_GLL0  in remapped NeCDF file
EASYMORE will remap variable  LCDC_P0_L1_GLL0  from source file to variable  LCDC_P0_L1_GLL0  in remapped NeCDF file
EASYMORE will remap variable  MCDC_P0_L1_GLL0  from source file to variable  MCDC_P0_L1_GLL0  i

### 3. Package into clean per-dam dataframes

In [6]:
import datetime
from tqdm import tqdm
from merit_process import *
import os,pickle
from config import basin_rain_path, basin_temp_path

def get_values(path):
    if os.path.isfile(path):
        df = pd.read_csv(path)#.set_index("ID")
        return df.iloc[3:6,1:].set_index("Unnamed: 1").values
    else:
        return -9999 * np.ones((3, 766))
    
def generate_dates(frm_year=2010, frm_month=1, frm_day=11,
                   to_year=2021, to_month=1, to_day=1,
                   delta=datetime.timedelta(hours=3)):
    """
    """
    interval = 0
    current_date = datetime.datetime(year=frm_year, month=frm_month, day=frm_day)
    end_date = datetime.datetime(year=to_year, month=to_month, day=to_day)
    date_list = []
    while current_date < end_date:
        current_date = current_date + delta
        date_list.append(current_date)
    return date_list

def datetime_to_str(x):
    """
    """
    x = [x.year, x.month, x.day, x.hour]
    x = [str(y) for y in x]
    return "_".join(["0"+y if len(y)==1 else y for y in x])

def extract_values(indir, variable, dates):
    """
        TO BE WRITTEN CORRECTLY:
        Get the correct path to the csv file.
    """
    values, timestamps = [],[]
    for date in tqdm(dates):
        path = indir + variable + "/" + datetime_to_str(date) + "_00-15.csv"
        values.append(get_values(path))
        timestamps.extend([(date + datetime.timedelta(hours=x)).timestamp() \
                           for x in range(3)])
    columns = gpd.read_file(dam_basins_path)["COMID"].values.tolist()
    timestamps_ = np.array(timestamps)
    return pd.DataFrame(np.concatenate(values), columns=columns,
                        index=pd.DatetimeIndex(timestamps_*10**9))

In [8]:
frm_year, frm_month, frm_day = 2010, 1, 1
to_year, to_month, to_day = 2020,1,1

dates = generate_dates(frm_year, frm_month, frm_day,
                       to_year, to_month, to_day)
hours = ['00', '03', '06', '09', '12', '15', '18', '21']

In [ ]:
indir = "/media/tristan/Elements/Hydro/weather/MSM_basin_var/"
outdir = "/media/tristan/Elements/Hydro/weather/MSM_basin_df/"

for variable in varin + sup_var:
    df = extract_values(indir, variable, dates)
    df.to_csv(outdir + variable + ".csv")